### STEP : Diversity Analysis



#### Example

- [“Moving Pictures” tutorial](https://docs.qiime2.org/2022.8/tutorials/moving-pictures-usage/?highlight=ancom)
- [Biostats ANCOM](http://scikit-bio.org/docs/0.4.2/generated/generated/skbio.stats.composition.ancom.html)


#### Methods
- [composition](https://docs.qiime2.org/2022.8/plugins/available/composition/)
- [composition add_pseudocount](https://docs.qiime2.org/2022.8/plugins/available/composition/add-pseudocount/): Increment all counts in table by pseudocount.
- [composition ancom](https://docs.qiime2.org/2022.8/plugins/available/composition/ancom/): Apply ANCOM to identify features that differ in abundance.

## Setup and settings

In [36]:
# Importing packages
import os
import pandas as pd
from qiime2 import Artifact
from qiime2 import Visualization
from qiime2 import Metadata

from qiime2.plugins.composition.visualizers import ancom
from qiime2.plugins.composition.methods import add_pseudocount

from qiime2.plugins.feature_table.methods import filter_samples
from qiime2.plugins.feature_table.methods import filter_seqs
from qiime2.plugins.feature_table.methods import relative_frequency

from qiime2.plugins.taxa.methods import collapse

import biom

%matplotlib inline

### Receiving the parameters

The following cell can receive parameters using the [papermill](https://papermill.readthedocs.io/en/latest/) tool.

In [37]:
metadata_file = '/mnt/nupeb/rede-micro/redemicro-ana-flavia-nutri/data/raw/metadata/metadata-ana-flavia-NCxNR.tsv'
base_dir = os.path.join('/', 'home', 'lauro', 'nupeb', 'rede-micro', 'redemicro-miliane-nutri')
base_dir = '/mnt/nupeb/rede-micro/redemicro-ana-flavia-nutri'
experiment_name = 'miliane-CxAC-trim'
class_col = 'group-id'
replace_files = False

In [38]:
experiment_folder = os.path.abspath(os.path.join(base_dir, 'experiments', experiment_name))
img_folder = os.path.abspath(os.path.join(experiment_folder, 'imgs'))

### Defining names, paths and flags

In [48]:
# QIIME2 Artifacts folder
qiime_folder = os.path.join(experiment_folder, 'qiime-artifacts')

# Input - DADA2 Artifacts
dada2_tabs_path = os.path.join(qiime_folder, 'dada2-tabs.qza')

In [50]:
qiime_folder
!pwd

/mnt/nupeb/rede-micro/redemicro-ana-flavia-nutri/nb-templates


## Step execution

### Load input files

This Step import the QIIME2 `FeatureTable[Frequency]` Artifact and the `Metadata` file.

In [44]:
metadata_file
!ls ../data/raw/metadata

metadata-ana-flavia-HSD-NCxHSD-NR.tsv  metadata-ana-flavia-NRxSTD-NR.tsv
metadata-ana-flavia-NCxHSD-NC.tsv      metadata-ana-flavia-STD-NCxHSD-NC.tsv
metadata-ana-flavia-NCxNR.tsv	       metadata-ana-flavia-STD-NCxSTD-NR.tsv
metadata-ana-flavia-NCxSTD-NC.tsv      metadata-ana-flavia-STD-NRxHSD-NR.tsv
metadata-ana-flavia-NRxHSD-NR.tsv


In [51]:
#Load Metadata
metadata_qa = Metadata.load(metadata_file)

#Load FeatureTable[Frequency]
tabs = Artifact.load(dada2_tabs_path)

# Filter FeatureTable[Frequency | RelativeFrequency | PresenceAbsence | Composition] based on Metadata sample ID values
tabs = filter_samples(
    table=tabs,
    metadata=metadata_qa,
).filtered_table

ValueError: /mnt/nupeb/rede-micro/redemicro-ana-flavia-nutri/experiments/miliane-CxAC-trim/qiime-artifacts/dada2-tabs.qza does not exist.

# ANCOM

Apply Analysis of Composition of Microbiomes (ANCOM) to identify features
that are differentially abundant across groups.

- [composition add_pseudocount](https://docs.qiime2.org/2022.8/plugins/available/composition/add-pseudocount/): Increment all counts in table by pseudocount.
- [composition ancom](https://docs.qiime2.org/2022.8/plugins/available/composition/ancom/): Apply ANCOM to identify features that differ in abundance.

In [ ]:
# Select class column
column = metadata_qa.get_column(class_col)

In [ ]:
# Create the pseudocount table
composition_tab = add_pseudocount(table = tabs).composition_table

In [ ]:
# Create visualizations for each parameter combination for the ANCOM  method.
transform_functions = ('sqrt', 'log', 'clr')
difference_functions = ('mean_difference', 'f_statistic')
for t in transform_functions:
    for d in difference_functions:
        print(f"Calculating ANCOM with: {t} {d}")
        try:
            ancom_viz = ancom(
                table = composition_tab, 
                metadata = column, 
                transform_function = t, 
                difference_function = d
            ).visualization
            view_name = os.path.join(qiime_folder, f'ancom-{t}-{d}.qzv')
            ancom_viz.save(view_name)
        except Exception as e:
            print(f"ERROR: Calculating ANCOM with: {t} {d}")
            print(e)

# LEfSe

LEfSe (Linear discriminant analysis Effect Size) determines the features (organisms, clades, operational taxonomic units, genes, or functions) most likely to explain differences between classes by coupling standard tests for statistical significance with additional tests encoding biological consistency and effect relevance.

- https://huttenhower.sph.harvard.edu/lefse/
- https://github.com/statonlab/BiGG2020_CrackNAg/wiki/qiime2-to-lefse
- https://github.com/biobakery/biobakery/wiki/lefse#2-lefse--conda-docker-vm-